In [58]:
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import copy
import pkg_resources
from numpy import ndarray
from numpy.typing import NDArray
from dataclasses import dataclass
from typing import Callable, List, Optional


In [60]:
from trachoma.trachoma_functions import *
import multiprocessing
from joblib import Parallel, delayed
import pkg_resources
num_cores = multiprocessing.cpu_count()

#############################################################################################################################
#############################################################################################################################

# initialize parameters, sim_params, and demography

params = {'N': 10000,
          'av_I_duration' : 2,
          'av_ID_duration':200/7,
          'inf_red':0.45,
          'min_ID':11, #Parameters relating to duration of infection period, including ID period
          'av_D_duration':300/7,
          'min_D':1, #Parameters relating to duration of disease period
          'v_1':1,
          'v_2':2.6,
          'phi':1.4,
          'epsilon':0.5,#Parameters relating to lambda function- calculating force of infection
          #Parameters relating to MDA
          'MDA_Cov':0.8,
          'MDA_Eff': 0.85, # Efficacy of treatment
          'rho':0.3,
          'nweeks_year':52,
          'babiesMaxAge':0.5, #Note this is years, need to check it converts to weeks later
          'youngChildMaxAge':9,#Note this is years, need to check it converts to weeks later
          'olderChildMaxAge':15, #Note this is years, need to check it converts to weeks later
          'b1':1,#this relates to bacterial load function
          'ep2':0.114,
          'n_inf_sev':38,
          'TestSensitivity': 0.96,
          'TestSpecificity': 0.965,
          'SecularTrendIndicator': 0,
          'SecularTrendYearlyBetaDecrease': 0.01,
          'vacc_prob_block_transmission':  0.6, 
          'vacc_reduce_bacterial_load': 1, 
          'vacc_reduce_duration': 1,
          'vacc_coverage': 0,  
          'vacc_waning_length': 52 * 5}


burnin = 100*52
timesim = burnin + 31*52

sim_params = {'timesim': timesim, 
              'burnin': burnin,
              'N_MDA':5, # irrelevant due to input method
              'n_sim':100}


demog = {'tau': 0.0004807692, 
         'max_age': 3120,
         'mean_age': 1040}



previous_rounds = 0


Start_date = date(2010,1, 1)
End_date = date(2040,12,31)


## Modifying Matt's updated run folder to include the version of sim_Ind_MDA which includes vaccination. Need to update/match inputs for SimulationFunction.

In [61]:
def seed_to_state(seed):
    np.random.seed(seed)
    return np.random.get_state()

outputYear = range(2019, 2041) # CHECK this, was 2019 to 2041
outputTimes = getOutputTimes(outputYear)
outputTimes = get_Intervention_times(outputTimes, Start_date, sim_params['burnin'])

# this is essentially the same as the original version of the Trachoma_Simulation function

def SimulationFunction(params, sim_params, demog, MDA_times, MDAData, vacc_times, VaccData, seed_bump, beta):

    # Longitudinal simulations:

    # Setting beta; the higher this is the higher the prevalence. 0.12 has prevalence around 20%
    #bet = np.random.uniform(size=sim_params['n_sim'], low=0.1, high=0.12)
    bet = np.ones(sim_params['n_sim']) * beta
    # Run multiple simulations
    def multiple_simulations(i):
        seed = i * seed_bump

        # Generate some random numbers
        np.random.seed(seed)
# we generate a numpy state for each simulation by saving a state. If the seed is set above, this will be consistent from run to run
        numpy_states = list(map(lambda s: seed_to_state(s), np.random.randint(2^32, size=1)))
        vals = Set_inits(params=params, demog=demog, sim_params = sim_params, MDAData=MDAData, numpy_state = numpy_states[0])    # Set initial conditions
        vals = Seed_infection(params=params, vals=vals) # Seed infection
        
        # vals = Check_and_init_MDA_and_survey_counts(vals, numpy_states[0])
        
        vals = Check_and_init_vaccination_state(params,vals)
        
        vals = Check_and_init_MDA_treatment_state(params, vals, MDAData, numpy_state=numpy_states[0])
        
        out, results = sim_Ind_MDA_Include_Survey(params, vals, timesim, burnin, demog, bet[i], MDA_times, MDAData, vacc_times, VaccData, outputTimes, doSurvey = False, doIHMEOutput = False, numpy_state = numpy_states[0])
        return out

    data_store_all_sim = Parallel(n_jobs=num_cores)(delayed(multiple_simulations)(i) for i in range(sim_params['n_sim']))

    return data_store_all_sim




# analyse the runs of the simulations
# we return some aggregated results along with the raw data
def analyseResults(data, sim_params):

    True_Prev_Infection_children_1_9 = np.zeros(shape=(sim_params['timesim'] , sim_params['n_sim']))
    True_Prev_Disease_children_1_9 = np.zeros(shape=(sim_params['timesim'] , sim_params['n_sim']))
    #True_Prev_Disease = np.zeros(shape=(sim_params['timesim'] , sim_params['n_sim']))
    #True_Prev_Infection = np.zeros(shape=(sim_params['timesim'] , sim_params['n_sim']))
    Time = np.arange(sim_params['timesim'] )

    
    for i in range(sim_params['n_sim']):

        #True_Prev_Disease[:, i] = data[i]['True_Prev_Disease'][0: sim_params['timesim']]
        #True_Prev_Infection[:, i] = data[i]['True_Prev_Infection'][0: sim_params['timesim']]
        True_Prev_Infection_children_1_9[:, i] = data[i]['True_Prev_Infection_children_1_9'][0: sim_params['timesim']]
        True_Prev_Disease_children_1_9[:, i] = data[i]['True_Prev_Disease_children_1_9'][0: sim_params['timesim']]

    
    results = pd.DataFrame({'Time': Time / 52,
                            'Mean_Disease_Children': np.mean(True_Prev_Disease_children_1_9, axis=1),
                            'Mean_Infection_Children': np.mean(True_Prev_Infection_children_1_9, axis=1),
                            #'Mean_Disease_All': np.mean(True_Prev_Disease, axis=1),
                            #'Mean_Infection_All': np.mean(True_Prev_Infection, axis=1),
                            'Median_Disease_Children': np.median(True_Prev_Disease_children_1_9, axis=1),
                            'Median_Infection_Children': np.median(True_Prev_Infection_children_1_9, axis=1),
                            #'Median_Disease_All': np.median(True_Prev_Disease, axis=1),
                            #'Median_Infection_All': np.median(True_Prev_Infection, axis=1)
                            })

    return results,True_Prev_Infection_children_1_9, True_Prev_Disease_children_1_9 #, True_Prev_Disease, True_Prev_Infection, 




# function to get the MDA data for the specified coverage file
def get_MDA_data(coverageFileName):
    MDAData = readPlatformData(coverageFileName, "MDA")
    MDA_dates = getInterventionDates(MDAData)
    MDA_times = get_Intervention_times(MDA_dates, Start_date, sim_params['burnin'])
    return MDAData, MDA_times

# function to get the vaccination data for the specified coverage file
def get_vacc_data(coverageFileName):
    VaccData = readPlatformData(coverageFileName, "Vaccine")
    vacc_dates = getInterventionDates(VaccData)
    vacc_times = get_Intervention_times(vacc_dates, Start_date, sim_params['burnin'])
    return VaccData, vacc_times

In [62]:

seed = 100
beta = 0.22


## Scenarios chosen by Ellie and Claudio

In [63]:
# loading in each scenario - currently just two and a baseline
MDADataBL, MDA_timesBL = get_MDA_data("scen_ellie_bl.csv")
VaccDataBL, vacc_timesBL = get_vacc_data("scen_ellie_bl.csv")

MDADataCW, MDA_timesCW = get_MDA_data("scen_ellie_cw.csv")
VaccDataCW, vacc_timesCW = get_vacc_data("scen_ellie_cw.csv")

MDADataCO, MDA_timesCO = get_MDA_data("scen_ellie_co.csv")
VaccDataCO, vacc_timesCO = get_vacc_data("scen_ellie_co.csv")

In [64]:
res_el = [[] for _ in range(9)]

In [ ]:
# runs 0-4

data_el_0 = SimulationFunction(params, sim_params, demog, MDA_timesBL, MDADataBL, vacc_timesBL, VaccDataBL, seed_bump = seed, beta = beta)
res_el[0] = analyseResults(data_el_0, sim_params)

params['vacc_prob_block_transmission'] = 0.5
params['vacc_waning_length'] = 1 * 52

data_el_1 = SimulationFunction(params, sim_params, demog, MDA_timesCO, MDADataCO, vacc_timesCO, VaccDataCO, seed_bump = seed, beta = beta)
res_el[1] = analyseResults(data_el_1, sim_params)

data_el_2 = SimulationFunction(params, sim_params, demog, MDA_timesCW, MDADataCW, vacc_timesCW, VaccDataCW, seed_bump = seed, beta = beta)
res_el[2] = analyseResults(data_el_2, sim_params)

params['vacc_waning_length'] = 15 * 52

data_el_3 = SimulationFunction(params, sim_params, demog, MDA_timesCO, MDADataCO, vacc_timesCO, VaccDataCO, seed_bump = seed, beta = beta)
res_el[3] = analyseResults(data_el_3, sim_params)

data_el_4 = SimulationFunction(params, sim_params, demog, MDA_timesCW, MDADataCW, vacc_timesCW, VaccDataCW, seed_bump = seed, beta = beta)
res_el[4] = analyseResults(data_el_4, sim_params)

In [ ]:
# runs 5-8

params['vacc_prob_block_transmission'] = 0.9
params['vacc_waning_length'] = 1 * 52

data_el_5 = SimulationFunction(params, sim_params, demog, MDA_timesCO, MDADataCO, vacc_timesCO, VaccDataCO, seed_bump = seed, beta = beta)
res_el[5] = analyseResults(data_el_5, sim_params)

data_el_6 = SimulationFunction(params, sim_params, demog, MDA_timesCW, MDADataCW, vacc_timesCW, VaccDataCW, seed_bump = seed, beta = beta)
res_el[6] = analyseResults(data_el_6, sim_params)

params['vacc_waning_length'] = 15 * 52

data_el_7 = SimulationFunction(params, sim_params, demog, MDA_timesCO, MDADataCO, vacc_timesCO, VaccDataCO, seed_bump = seed, beta = beta)
res_el[7] = analyseResults(data_el_7, sim_params)

data_el_8 = SimulationFunction(params, sim_params, demog, MDA_timesCW, MDADataCW, vacc_timesCW, VaccDataCW, seed_bump = seed, beta = beta)
res_el[8] = analyseResults(data_el_8, sim_params)

In [ ]:
import matplotlib.pyplot as plt
start_year = 2010-burnin/52 # was 2018 - check to match csv dates

scenarios = [0,1,2,3,4,5,6,7,8]

for i in scenarios:
    plt.plot(start_year + np.array(res_el[0][0]['Time'][(sim_params['burnin']-100): sim_params['timesim']]),np.array(res_el[i][0]['Median_Disease_Children'][(sim_params['burnin']-100): sim_params['timesim']]))
    
plt.legend(['Scen' + str(i) for i in range(9)]) 
plt.title("Prevalence of disease in children 1-9")
plt.xlim([2018,2042])

In [ ]:
scenarios = [0,1,2,3,4,5,6,7,8]

for i in scenarios:
    plt.plot(start_year + np.array(res_el[0][0]['Time'][(sim_params['burnin']-100): sim_params['timesim']]),np.array(res_el[i][0]['Median_Infection_Children'][(sim_params['burnin']-100): sim_params['timesim']]))
    
plt.legend(['Scen' + str(i) for i in range(9)]) 
plt.title("Prevalence of infection in children 1-9")
plt.xlim([2018,2042])

## Export

In [53]:
#scenM1 = res_no_interruption[0]['Median_Infection_Children']
#scenM2 = res_interrupt_no_mitigation[0]['Median_Infection_Children']
#scenM3 = res_interrupt_mitigation[0]['Median_Infection_Children']

In [54]:
'''
exportmatt = pd.DataFrame({'scenM1': scenM1,
                          'scenM2': scenM2,
                          'scenM3': scenM3})

SyntaxError: incomplete input (1888117950.py, line 1)

In [ ]:
#exportmatt.to_csv('matt.csv', index=True)